In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
# from datetime import datetime
# from dateutil.parser import parse
# from utilityFunctions import *

In [2]:
# working directory
working_dir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/NewMexico/AggregatedAmounts/oldcode"
os.chdir(working_dir)

In [3]:
target_columns = ["ReportingUnitUUID", "ReportingUnitNativeID", "ReportingUnitName", 
                  "ReportingUnitTypeCV", "ReportingUnitUpdateDate", "ReportingUnitProductVersion",
                  "StateCV", "EPSGCodeCV", "Geometry"]

In [4]:
outdf100 = pd.DataFrame(columns=target_columns)

In [5]:
# Input files
fileInput1 = "RawInputData/Summary of withdrawals by county 90-15.xlsx" 
# fileInput2 = "Summary of withdrawals by River Basin 90-15.xlsx" 

In [6]:
df10 = pd.read_excel(fileInput1, header=0, sheet_name=0, skiprows=1, encoding = "ISO-8859-1")
#df20 = pd.read_excel(fileInput2, header=0, sheet_name=0, skiprows=1, encoding = "ISO-8859-1")
list(df10.columns)
#list(df20.columns)

FileNotFoundError: [Errno 2] No such file or directory: 'RawInputData/Summary of withdrawals by county 90-15.xlsx'

In [7]:
#Extracting data from multiple sheets.  Each sheet is its own ReportYear.
startYear = 1990
endYear = 2015
numSheets = 5
yearList = np.linspace(startYear, endYear, numSheets)
df100_list = []
for isx in range (numSheets):
    df10 = pd.read_excel(fileInput1, header=0, sheet_name=isx, skiprows=1, encoding = "ISO-8859-1")
    df10 = df10.assign(ReportYearCV=yearList[isx])
    df10.ReportYearCV = df10.ReportYearCV.astype(int)
    df100_list.append(df10)
    
df100 = pd.concat(df100_list, sort=True)

df100

,CAT,CN,COUNTY,ReportYearCV,TW,WGW,WSW
0,Public Water Supply,1,Bernalillo,1990,125483.156250,125483.156250,0.000000
1,Domestic (self-supplied),1,Bernalillo,1990,3561.899902,3561.899902,0.000000
2,Irrigated Agriculture,1,Bernalillo,1990,77764.000000,4037.000000,73727.000000
3,Livestock (self-supplied),1,Bernalillo,1990,789.530029,753.200012,36.330002
4,Commercial (self-supplied),1,Bernalillo,1990,3711.300049,3711.300049,0.000000
...,...,...,...,...,...,...,...
292,Livestock (self-supplied),61,Valencia,2015,888.255259,841.114994,47.140265
293,Mining (self-supplied),61,Valencia,2015,178.559000,178.559000,0.000000
294,Power (self-supplied),61,Valencia,2015,6.000000,6.000000,0.000000
295,Public Water Supply,61,Valencia,2015,6553.686693,6553.686693,0.000000


In [8]:
print("Copying County name...")

# all we need from input is county name

# first remove redundancies
df100 = df100.drop_duplicates(subset=["COUNTY"])   #
df100 = df100.reset_index(drop=True)

print(len(df100.index))

outdf100["ReportingUnitName"] = df100["COUNTY"]

Copying County name...
33


In [13]:
#ReportingUnitNativeID
#based off the GEOID we have in our shape files.

nameGEOIDDict = {
"Bernalillo" : "35001",
"Catron" : "35003",
"Chaves" : "35005",
"Cibola" : "35006",
"Colfax" : "35007",
"Curry" : "35009",
"De Baca" : "35011",
"Dona Ana" : "35013",
"Eddy" : "35015",
"Grant" : "35017",
"Guadalupe" : "35019",
"Harding" : "35021",
"Hidalgo" : "35023",
"Lea" : "35025",
"Lincoln" : "35027",
"Los Alamos" : "35028",
"Luna" : "35029",
"McKinley" : "35031",
"Mora" : "35033",
"Otero" : "35035",
"Quay" : "35037",
"Rio Arriba" : "35039",
"Roosevelt" : "35041",
"San Juan" : "35045",
"San Miguel" : "35047",
"Sandoval" : "35043",
"Santa Fe" : "35049",
"Sierra" : "35051",
"Socorro" : "35053",
"Taos" : "35055",
"Torrance" : "35057",
"Union" : "35059",
"Valencia" : "35061"
}


def retrieveGEOID(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = colrowValue
    else:
        String1 = colrowValue  # remove whitespace chars
        try:
            outList = nameGEOIDDict[String1]
        except:
            outList = colrowValue
    return outList

outdf100['ReportingUnitNativeID'] = df100.apply(lambda row: retrieveGEOID(row['COUNTY']), axis=1)
outdf100

,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,ReportingUnitUpdateDate,ReportingUnitProductVersion,StateCV,EPSGCodeCV,Geometry
0,35001,35001,Bernalillo,NaN,NaN,NaN,NaN,NaN,NaN
1,35003,35003,Catron,NaN,NaN,NaN,NaN,NaN,NaN
2,35005,35005,Chaves,NaN,NaN,NaN,NaN,NaN,NaN
3,35006,35006,Cibola,NaN,NaN,NaN,NaN,NaN,NaN
4,35007,35007,Colfax,NaN,NaN,NaN,NaN,NaN,NaN
5,35009,35009,Curry,NaN,NaN,NaN,NaN,NaN,NaN
6,35011,35011,De Baca,NaN,NaN,NaN,NaN,NaN,NaN
7,35013,35013,Dona Ana,NaN,NaN,NaN,NaN,NaN,NaN
8,35015,35015,Eddy,NaN,NaN,NaN,NaN,NaN,NaN
9,35017,35017,Grant,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# # ReportingUnitNativeID	 	 Auto generate
# outdf100['ReportingUnitNativeID'] = range(1, len(outdf100.index) + 1)

#ReportingUnitUUID	 	 NM_NativeID
outdf100['ReportingUnitUUID'] = outdf100.apply(lambda row: "_".join(["NM_C", str(row['ReportingUnitNativeID'])]), axis=1)
outdf100

,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,ReportingUnitUpdateDate,ReportingUnitProductVersion,StateCV,EPSGCodeCV,Geometry
0,NM_C_35001,35001,Bernalillo,NaN,NaN,NaN,NaN,NaN,NaN
1,NM_C_35003,35003,Catron,NaN,NaN,NaN,NaN,NaN,NaN
2,NM_C_35005,35005,Chaves,NaN,NaN,NaN,NaN,NaN,NaN
3,NM_C_35006,35006,Cibola,NaN,NaN,NaN,NaN,NaN,NaN
4,NM_C_35007,35007,Colfax,NaN,NaN,NaN,NaN,NaN,NaN
5,NM_C_35009,35009,Curry,NaN,NaN,NaN,NaN,NaN,NaN
6,NM_C_35011,35011,De Baca,NaN,NaN,NaN,NaN,NaN,NaN
7,NM_C_35013,35013,Dona Ana,NaN,NaN,NaN,NaN,NaN,NaN
8,NM_C_35015,35015,Eddy,NaN,NaN,NaN,NaN,NaN,NaN
9,NM_C_35017,35017,Grant,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# hardcoded

outdf100.StateCV = "NM"
outdf100.ReportingUnitTypeCV = "County"
outdf100.EPSGCodeCV = "EPSG:4326"
sampleWKT =  'POLYGON((-99.54319297853704 37.15853229006052, -97.26976797641987 37.15759429005948, -105.11636298372741 37.14764529005038, -104.52740598317905 37.15119229005359, -104.09963198278069 37.15376929005606, -103.56062798227867 37.156443290058405, -103.12301898187116 37.157137290059154, -103.08639398183686 37.15689329005886, -103.00203898175846 37.156332290058344, -99.90287697887197 37.162385290064094, -99.54319297853704 37.15853229006052))'
outdf100.Geometry = sampleWKT

# replace NaN with blank cells
outdf100 = outdf100.replace(np.nan, '')
outdf100.head(5)

,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,ReportingUnitUpdateDate,ReportingUnitProductVersion,StateCV,EPSGCodeCV,Geometry
0,NM_C_35001,35001,Bernalillo,County,,,NM,EPSG:4326,"POLYGON((-99.54319297853704 37.15853229006052,..."
1,NM_C_35003,35003,Catron,County,,,NM,EPSG:4326,"POLYGON((-99.54319297853704 37.15853229006052,..."
2,NM_C_35005,35005,Chaves,County,,,NM,EPSG:4326,"POLYGON((-99.54319297853704 37.15853229006052,..."
3,NM_C_35006,35006,Cibola,County,,,NM,EPSG:4326,"POLYGON((-99.54319297853704 37.15853229006052,..."
4,NM_C_35007,35007,Colfax,County,,,NM,EPSG:4326,"POLYGON((-99.54319297853704 37.15853229006052,..."


In [17]:
print("Writing out...")

#write out
out_repunit = 'ProcessedInputData/reportingunits.csv'
outdf100.to_csv(out_repunit, index=False, encoding = "utf-8")

print("Done sites")

Writing out...
Done sites
